In [15]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
!pip install langchain-groq

In [5]:
VISION_MODEL_NAME="llama-3.2-11b-vision-preview"

In [34]:
TEMPLATE="""
Eres un experto storyteller
en particular 
{topic}

El formato de salida debe ser el siguiente:{format_instructions}
"""

In [35]:
class ContentGenerationScript(BaseModel):
    text: str = Field(..., description="Contenido textual de la historia")

parser= JsonOutputParser(pydantic_object=ContentGenerationScript)

In [36]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192",
    groq_api_key="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"
)


prompt = PromptTemplate(
    input_variables=["topic"],
    template=TEMPLATE,
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


chain = LLMChain(llm=llm, prompt=prompt)


result = chain.invoke({"topic": "Ironman"})


result

{'topic': 'Ironman',
 'text': '¡Excelente! Como experto en storytelling de Iron Man, te presento la siguiente historia en formato JSON:\n\n```\n{\n  "text": "Tony Stark, el genio detrás de Iron Man, se despertó en su habitación de hospital con una sensación de confusión y dolor. Recordaba vagamente el accidente en el que su cohete espacial se estrelló en el Pacífico, pero no podía recordar cómo había sobrevivido. Mientras intentaba recordar los detalles, escuchó la voz de Pepper Potts, su secretaria y amiga, que le decía que había sido rescatado y que estaba en el hospital. Tony se dio cuenta de que había perdido la memoria y que su identidad como Iron Man estaba en peligro. Decidió investigar y descubrir la verdad sobre su pasado y su identidad, pero pronto se dio cuenta de que no era el único que buscaba la verdad..."\n}\n```\n\nEspero que disfrutes de esta historia en formato JSON. ¡Si necesitas más, no dudes en preguntar!'}

In [37]:
import base64
from io import BytesIO
from PIL import Image
import os

import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from groq import Groq

def encode_image(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def describe_image(image_path):
    
    VISION_MODEL_NAME="llama-3.2-11b-vision-preview"
    client = Groq(api_key='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')
    vision_model = VISION_MODEL_NAME
    
    image = Image.open(image_path)
    base64_image = encode_image(image)
    
    completion = client.chat.completions.create(
        model=vision_model,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Describe la imagen a detalle en espanol"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
        temperature=1,
        max_completion_tokens=1024,
    )
    
    return completion.choices[0].message.content




In [39]:
image_path = "C:/Users/jcama/Desktop/rolex.jpg" 
description = describe_image(image_path)
print("Image description:", description)

Image description: Bien, se nos muestra un reloj de suelo de dos tonos. Por lo tanto, la imagen que estamos buscando se refiere a un reloj hecho en dos colores, solo de uno de los cuales es visible en esta foto. La imagen probablemente se muestra desde un ángulo, ya que la parte final de las dos agujas de la cara del reloj no está visible. En el centro del reloj, se muestra «Rolex» en minúsculas. El dial negro tiene letras y números, todos rematados o dibujados en blanco, desde la posición de la manecilla de las horas hasta un punto del reloj, y otra de las manecillas de alfiler que toca «Rolex». Estas 4 letras se encuentran en el centro de la imagen en el lado derecho del reloj. 

La banda del reloj es en una posición de suelo donde se muestra la parte superior de su parte central, el doblado de la banda y toda su extensión, es paralela a la línea que conecta los dos puntos de la imagen. 
Esto hace que la mitad superior del reloj sea visible, de la manecilla de las horas hasta toda la